GETTING DATA FROM API's

GOALS
1. Extract stock data from AlphaVantage API using HTTP requests.
2. Write a function for transforming stock data

In [12]:
import pandas as pd
import requests

Accesing API's through URL

In [13]:
api_key = "PGPVDW57277TSFM3"

URL for Alibaba Limited Company

In [14]:
ticker = "BABA"
output_size = "compact"
data_type = "json"

url = (
    "https://www.alphavantage.co/query?"
    "function=TIME_SERIES_DAILY&"
    f"symbol={ticker}&"
    f"outputsize={output_size}&"
    f"datatype={data_type}&"
    f"apikey={api_key}"
)
print("url type:", type(url))
url

url type: <class 'str'>


'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=BABA&outputsize=compact&datatype=json&apikey=PGPVDW57277TSFM3'

Accessing API's through a Request

In [15]:
response = requests.get(url=url)
print("response type:", type(response))

response type: <class 'requests.models.Response'>


Response Attributes

In [16]:
dir(response)

['__attrs__',
 '__bool__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__nonzero__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_content',
 '_content_consumed',
 '_next',
 'apparent_encoding',
 'close',
 'connection',
 'content',
 'cookies',
 'elapsed',
 'encoding',
 'headers',
 'history',
 'is_permanent_redirect',
 'is_redirect',
 'iter_content',
 'iter_lines',
 'json',
 'links',
 'next',
 'ok',
 'raise_for_status',
 'raw',
 'reason',
 'request',
 'status_code',
 'text',
 'url']

Response Status Code

In [17]:
print(response.status_code)

200


Response JSON

In [26]:
response_data = response.json()
print("response_json type:", type(response_data))

response_json type: <class 'dict'>


Response Data Keys

In [27]:
response_data.keys()

dict_keys(['Information'])

In [28]:
stock = response_data["Information"]
stock

'Thank you for using Alpha Vantage! Our standard API rate limit is 25 requests per day. Please subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly remove all daily rate limits.'

Get Daily Function

In [20]:
def get_daily(ticker, output_size="full"):
    """Get daily time series of an equity from AlphaVantage.
    parameters
    _ _ _ _ _ _
    ticker: str
        The ticker symbol of the equity.
    output_size: str
        Number of observations to return. "full" means the historic observations of the equtiy to be returned, "compact means the most recent observations."
    returns
    _ _ _ _ _ _
    pd.DataFrame:
        Columns are "open, high, low, close, volume"
    """
    
    # Create URL
    url = (
        "https://www.alphavantage.co/query?"
        "function=TIME_SERIES_DAILY&"
        f"symbol={ticker}&"
        f"outputsize={output_size}&"
        f"datatype=json&"
        f"apikey={api_key}"
    )
    
    # Send request to api
    response = requests.get(url=url)
    
    # Check if the request  is successfull
    if response.status_code != 200:
        raise Exception(f"Error fetching data: {response.status_code}")
    
    # Extract JSON data from response
    response_data = response.json()
    
    # Check if the "Time Series key exists" in response_data
    if "Time Series (Daily)" not in response_data.keys():
        raise Exception(
            f"Invalid API call. Check the ticker symbol: {ticker}"
        )
    
    # Extract json data from response
    data = response_data["Time Series (Daily)"]
    
    
    # Read data into DataFrame
    stock_data = pd.DataFrame.from_dict(data, orient="index", dtype="float")
    
    # Convert "index" to "datetime index" and name "date"
    stock_data.index =  pd.to_datetime(stock_data.index)
    stock_data.index.name = "date"

    # Removing numbering from your column names
    stock_data.columns = [c.split(".")[1] for c in stock_data.columns]
    
    return stock_data

Testing Get Daily Function

In [23]:
get_daily(ticker="BABA")

Exception: Invalid API call. Check the ticker symbol: BABA